In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os, sys
path = os.getcwd()
while not path.endswith('research'):    
    path = os.path.abspath(os.path.join(path, '../'))
sys.path.append(path)

In [3]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

***

In [4]:
import pandas as pd
from datetime import datetime

In [8]:
from gmt.fom.gas.shared.utils.common import  pa
from gmt.fom.gas.shared.utils.dframe import  ddff

from gmt.fom.gas.shared.utils.common import flatten_lst
from gmt.fom.gas.shared.data_sources.mongo import GasMongo

In [9]:
myMongo = GasMongo()

2021-03-17 12:08:59,954: c:\git\gmt-fom-gas-research\gmt\fom\gas\shared\data_sources\mongo.py: 25: DEBUG: Connected successfully.


In [10]:
#see databases
myMongo.databases

['GasModels']

In [11]:
myMongo.choose_db('GasModels')

In [12]:
#see collections in the database
myMongo.collections

['stpb_point_connect_historical_forecast',
 'stpb_gas_view_power_station_gas_demand',
 'stpb_covariates_temperature',
 'stpb_covariates_precipitation',
 'stpb_covariates_wind',
 'netback_variables',
 'netback_costs',
 'stpb_covariates_dark_spread',
 'netback_results',
 'stpb_covariates_clean_spread',
 'stpb_power_view_power_station_fuel_type_demand',
 'entsog_capacity']

In [12]:
myMongo.choose_collection('netback_costs')

In [13]:
myMongo.select_all()

Ballast - Speed (knots)  Days in port  \
0                       17           NaN   

   Fuel Share - Fuel oil for steam turbine  \
0                                        0   

   Fuel Share - LNG boil-off for steam turbine  \
0                                            1   

   Fuel consumption in port (tonnes/day)  Fuel oil consumption (tonnes/day)  \
0                                     25                                125   

   Heel (M3)  Heel (mmbtu)  Laden - Natural boil-off gas (%/day)  \
0        500         14000                                 0.001   

   Laden - Speed (knots)  ...  Pilot Light Fuel Cons (tonnes/day)  \
0                     17  ...                                   1   

   Ship Size (M3)  Spot charter rate (USD/day)  Suez - Ballast ($)  \
0          160000                            0         364443.5531   

   Suez - Laden ($)  Suez - Total ($)  USD/tonne - HFO  Unnamed: 1  \
0       428438.5939        792882.147              414         NaN   

   Unnamed: 10  Unnamed: 11  
0          NaN          NaN  

[1 rows x 29 columns]

***

In [13]:
myMongo.choose_collection('sandbox')

2021-03-17 12:09:17,703: c:\git\gmt-fom-gas-research\gmt\fom\gas\shared\data_sources\mongo.py: 48: INFO: Choosing collection sandbox.


In [107]:
df = pd.DataFrame([range(100)], columns=[f'a_{i}' for i in range(100)]).reindex()

In [108]:
for i in range(20):
    df = df.append(df)

In [109]:
myMongo.clear_collection()

In [103]:
myMongo.collection.insert_many(df.to_dict(orient='records'))

In [104]:
records = [r for r in myMongo.collection.find()]

In [105]:
df = pd.DataFrame(records).drop('_id',axis=1)

In [106]:
df

a_0  a_1  a_10  a_11  a_12  a_13  a_14  a_15  a_16  a_17  ...  a_90  a_91  \
0    0    1    10    11    12    13    14    15    16    17  ...    90    91   
1    0    1    10    11    12    13    14    15    16    17  ...    90    91   
2    0    1    10    11    12    13    14    15    16    17  ...    90    91   
3    0    1    10    11    12    13    14    15    16    17  ...    90    91   

   a_92  a_93  a_94  a_95  a_96  a_97  a_98  a_99  
0    92    93    94    95    96    97    98    99  
1    92    93    94    95    96    97    98    99  
2    92    93    94    95    96    97    98    99  
3    92    93    94    95    96    97    98    99  

[4 rows x 100 columns]

In [88]:
myMongo.select_all()

a_0  a_1  a_10  a_11  a_12  a_13  a_14  a_15  a_16  a_17  ...  a_90  a_91  \
0    0    1    10    11    12    13    14    15    16    17  ...    90    91   

   a_92  a_93  a_94  a_95  a_96  a_97  a_98  a_99  
0    92    93    94    95    96    97    98    99  

[1 rows x 100 columns]

In [15]:
myMongo.select_all().head()

Canal Fee  Distances  Port Charges  Regas Costs    RegionFrom TerminalFrom  \
0        No       4623   199309.8358         0.38  US/Caribbean       Sabine   
1        No       3156   213387.2208         0.38  US/Caribbean   Cove point   
2        No       4161   100830.4708         0.38      W Africa      Nigeria   
3        No       4620   112760.0333         0.38      W Africa       Angola   
4      Suez       6186   158119.3658         0.38   Middle East        Qatar   

  TerminalTo  
0  UK (Hook)  
1  UK (Hook)  
2  UK (Hook)  
3  UK (Hook)  
4  UK (Hook)

***

In [16]:
myMongo.choose_collection('netback_results')

In [20]:
myMongo.clear_collection()

In [21]:
myMongo.collection.count()

C:\Users\ashubert\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


0

In [40]:
df = pd.read_csv(r'C:\git\ARCHIVE\CHRIS_SCRIPTS\2021_01_15.csv')

In [41]:
def conv_dates_df(df, flag):
    if flag:
        df['PointDate_Origin'] = df['PointDate_Origin'].apply(lambda t: datetime.strptime(t, '%Y-%m-%d'))
        df['EffectiveDate'] = df['EffectiveDate'].apply(lambda t: datetime.strptime(t, '%Y-%m-%d'))
        df['PointDate_Destination'] = df['PointDate_Destination'].apply(lambda t: datetime.strptime(t, '%Y-%m-%d'))
    df['PointDate_Origin'] = df['PointDate_Origin'].apply(lambda t: pd.to_datetime(t))
    df['EffectiveDate'] = df['EffectiveDate'].apply(lambda t: pd.to_datetime(t))
    df['PointDate_Destination'] = df['PointDate_Destination'].apply(lambda t: pd.to_datetime(t))
    return df

In [42]:
df = conv_dates_df(df, True)

In [43]:
df['EffectiveDate'][0]

Timestamp('2021-01-15 00:00:00')

In [44]:
myMongo.pandas_to_mongo(df)

In [45]:
myMongo.collection.count()

C:\Users\ashubert\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


5040

In [47]:
res_df = myMongo.select_all()

In [48]:
res_df['EffectiveDate'].unique()

array(['2021-01-19T00:00:00.000000000', '2021-01-18T00:00:00.000000000',
       '2021-01-15T00:00:00.000000000'], dtype='datetime64[ns]')

In [59]:
t0 = datetime(2021,1,15)
t1 = datetime(2021,1,18)

query = {'EffectiveDate':{"$gte": t0, "$lte": t1}}

res_df2 = myMongo.select(query)

res_df2['EffectiveDate'].unique()